En este script, vamos a generar el envío de datos al puerto 9009 mediante una conexión TCP-IP. Estos datos serán recogidos y procesados por $Spark$ (en el otro script). 

En mi caso, voy a utilizar un corpus que contiene artículos en formato JSON publicados en 2016 (en concreto desde Enero hasta Junio). El procedimiento va a consistir en leer los artículos uno a uno, e ir mandándoselos a $Spark$ para que haga el tratamiento adecuado de cada uno. Vamos a comenzar planteando la lectura de dichos ficheros JSON.

In [5]:
import json
import time
from os import listdir

# Obtenemos los ficheros json contenidos en el directorio 2016
json_list = listdir('2016')
# print(json_list)

def read_json_and_send(tcp_connection):
    # Recorremos la lista de ficheros json
    for js in json_list:
        print(f"Enviando artículo {js}")
        # Usamos la libería json para cargarlo
        with open('./2016/' + js) as json_file:    
            data = json.load(json_file)[0]
            # De entre todos los campos del fichero json nos interesa "article" que contiene el grueso del texto del artítulo
            article = data['article']
            # Tomamos el texto leído y lo mandamos por nuestra conexión tcp a Spark (haciendo un encode para pasarlo a bytes)
            tcp_connection.sendall((article + '\n').encode('utf-8'))
        # Esperamos 1 segundo y pasamos a leer y mandar el siguiente artículo
        print(f"Artículo {js} enviado\n\n")
        time.sleep(1)        

La función read_json_and_send() recibe como argumento una conexión tcp y comienza a mandar uno a uno los ficheros json en el directorio "2016". De cada fichero, nos interesa su campo "article" que extraeremos y mandaremos al puerto 9009 en localhost. Entre cada artículo haremos una pausa de 1 segundo. 

In [6]:
import socket

TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
# Esperamos a una conexión TCP-IP al puerto 9009
conn, addr = s.accept()
# Cuando se establezca la conexión comenzamos a procesar los datos recibidos
print("Connected... Sending data")
# Pasamos a enviar uno a uno los artículos
read_json_and_send(conn)
conn.send("END".encode('utf-8'))
conn.close()

Waiting for TCP connection...
Connected... Sending data
Enviando artículo 2016-02-17.json
Artículo 2016-02-17.json enviado


Enviando artículo 2016-03-21.json
Artículo 2016-03-21.json enviado


Enviando artículo 2016-05-02.json
Artículo 2016-05-02.json enviado


Enviando artículo 2016-01-10.json
Artículo 2016-01-10.json enviado


Enviando artículo 2016-01-27.json
Artículo 2016-01-27.json enviado


Enviando artículo 2016-04-18.json
Artículo 2016-04-18.json enviado


Enviando artículo 2016-05-01.json
Artículo 2016-05-01.json enviado


Enviando artículo 2016-04-26.json
Artículo 2016-04-26.json enviado


Enviando artículo 2016-05-07.json
Artículo 2016-05-07.json enviado


Enviando artículo 2016-03-23.json
Artículo 2016-03-23.json enviado


Enviando artículo 2016-03-12.json
Artículo 2016-03-12.json enviado


Enviando artículo 2016-02-07.json
Artículo 2016-02-07.json enviado


Enviando artículo 2016-02-02.json
Artículo 2016-02-02.json enviado


Enviando artículo 2016-01-11.json
Artículo 2016